In [ ]:
import os
import numpy as np
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from underthesea import word_tokenize

# 1. Load data
df = pd.read_csv(r"../Datasets/jobs_cleaned.csv")
# Loại bỏ dòng trống để tránh lỗi mảng 0 sample
df = df.dropna(subset=['e5_input']).reset_index(drop=True)
docs = df['e5_input'].tolist()

# 2. Embedding với E5 (Dùng bản multilingual cho Anh-Việt)
embedding_model = SentenceTransformer('intfloat/multilingual-e5-large')
embeddings = embedding_model.encode(docs, show_progress_bar=True)

# 3. Cấu hình các thành phần (Fix lỗi ensure_all_finite bằng cách khởi tạo mới)
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
vectorizer_model = CountVectorizer(stop_words="english", min_df=2)

Batches: 100%|██████████| 105/105 [47:01<00:00, 26.87s/it]


In [ ]:
# Lưu models để sử dụng lại sau này
embedding_model.save(r"../Models/embedding_model")
embedding_path = r"../Models/embedding_model/embeddings.npy"
np.save(embedding_path, embeddings)

# Chạy lại 

In [2]:
import os
import numpy as np
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from underthesea import word_tokenize

df = pd.read_csv(r"../Datasets/jobs_cleaned.csv")
# Loại bỏ dòng trống để tránh lỗi mảng 0 sample
df = df.dropna(subset=['e5_input']).reset_index(drop=True)
docs = df['e5_input'].tolist()

print(f"Số lượng văn bản (docs): {len(docs)}")


Số lượng văn bản (docs): 3358


In [3]:
# Hàm tách từ tiếng Việt
def vi_tokenizer(text):
    # Tách từ và nối lại bằng dấu gạch dưới để CountVectorizer hiểu là 1 từ
    return word_tokenize(text, format="text")

print(" Đang tiến hành tách từ tiếng Việt...")
# Áp dụng tách từ cho toàn bộ docs
docs = [vi_tokenizer(doc) for doc in docs]

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
vectorizer_model = CountVectorizer(stop_words="english", min_df=2, token_pattern=r"(?u)\b\w\w+\b")
embedding_model = SentenceTransformer(r"../Models/embedding_model")
embeddings = embedding_model.encode(docs, show_progress_bar=True)
embedding_path = r"../Models/embedding_model/embeddings.npy"
np.save(embedding_path, embeddings)

 Đang tiến hành tách từ tiếng Việt...


Batches: 100%|██████████| 105/105 [50:48<00:00, 29.04s/it]


In [ ]:
# 4. Khởi tạo và chạy BERTopic
topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    nr_topics="auto"
)

topics, probs = topic_model.fit_transform(docs, embeddings)

# # 5. Tính toán phân bổ theo chuyên môn (Để dùng cho đoạn code dưới)
# topics_per_class = topic_model.topics_per_class(docs, classes=classes)

# # 6. Trực quan hóa
# fig_topics = topic_model.visualize_topics()
# fig_hierarchy = topic_model.visualize_hierarchy()
# fig_barchart = topic_model.visualize_barchart(top_n_topics=10)

# Lưu kết quả
topic_model.save(r"../Models/job_topic_model")
# fig_barchart.write_html(r"../Figures/topic_keywords.html")
# fig_class.write_html(r"../Figures/topics_by_job_role.html")


2026-01-09 19:35:08,325 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [5]:
# 1. Gán Topic ID vào DataFrame
df['Topic_ID'] = topics

# 2. Tạo hàm lấy 5 từ khóa cho mỗi Topic ID
# Lấy bảng thông tin chung của các topic
all_topics_info = topic_model.get_topics()

def get_top_10_words(topic_id):
    if topic_id == -1:
        return "Nhiễu/Không xác định"
    # Lấy danh sách từ khóa và chọn 5 từ đầu tiên
    words = all_topics_info[topic_id]
    top_10 = ", ".join([w[0] for w in words[:10]])
    return top_10

# 3. Tạo cột mới 'Topic_Keywords' dựa trên Topic_ID
# Tạo một từ điển map cho nhanh
topic_map = {topic_id: get_top_10_words(topic_id) for topic_id in set(topics)}
df['Topic_Keywords'] = df['Topic_ID'].map(topic_map)

# 4. Lưu lại dataset mới
df.to_csv(r"../Datasets/jobs_with_topics.csv", index=False, encoding="utf-8-sig")

print(" Đã gán Topic ID và 10 từ khóa vào dataset.")
print(df[['Tiêu đề', 'Topic_ID', 'Topic_Keywords']].head())

 Đã gán Topic ID và 10 từ khóa vào dataset.
               Tiêu đề  Topic_ID  \
0         Data Analyst         0   
1         Data Analyst         1   
2         Data Analyst         0   
3         Data Analyst         0   
4  Senior Data Analyst        -1   

                                      Topic_Keywords  
0  data, experience, business, skills, ai, work, ...  
1  dữ_liệu, phân_tích, kinh_nghiệm, làm_việc, côn...  
2  data, experience, business, skills, ai, work, ...  
3  data, experience, business, skills, ai, work, ...  
4                               Nhiễu/Không xác định  


In [6]:
# 1. Lấy bảng thông tin tổng quát về các topic
topic_info = topic_model.get_topic_info()

# 2. Lấy top 15 topic (loại bỏ Topic -1 vì đây là nhóm nhiễu/outliers)
top_15_topics = topic_info[topic_info["Topic"] != -1].head(15)

print(f"{'ID':<5} | {'Số lượng':<10} | {'Từ khóa đặc trưng cho Topic'}")
print("-" * 80)

# 3. Duyệt qua từng topic để in chi tiết
for probs, row in top_15_topics.iterrows():
    topic_id = row["Topic"]
    count = row["Count"]
    
    # Lấy danh sách từ khóa của topic này
    words_data = topic_model.get_topic(topic_id)
    
    if words_data:
        # Lấy 7 từ khóa đầu tiên để mô tả rõ nét hơn
        description = ", ".join([word[0] for word in words_data[:7]])
    else:
        description = "Không có dữ liệu"
        
    print(f"{topic_id:<5} | {count:<10} | {description}")

# --- Mẹo: Nếu bạn muốn lưu danh sách này ra file Excel để báo cáo ---
# top_15_topics.to_excel("../Datasets/top_15_topics_summary.xlsx", index=False)

ID    | Số lượng   | Từ khóa đặc trưng cho Topic
--------------------------------------------------------------------------------
0     | 880        | data, experience, business, skills, ai, work, management
1     | 747        | dữ_liệu, phân_tích, kinh_nghiệm, làm_việc, công_việc, để, khách_hàng
2     | 63         | hitachi, people, best, innovation, work, digital, look
3     | 48         | binance, blockchain, trading, ecosystem, arrangement, web3, worlds
4     | 46         | community, exam, assessment, complete, media, content, internet
5     | 40         | publicis, consulting, possible, data, 60, candidates, team
6     | 32         | qualcomm, engineering, research, related, individuals, systems, ai
7     | 32         | coding, language, tomorrows, expertise, pay, type, ai
8     | 27         | nvidia, ai, computing, gpu, wireless, future, models
9     | 27         | description, group, technical, colleagues, vietnam, database, 26
10    | 26         | asia, branches, united, bank,

In [7]:
# 1. Lấy toàn bộ thông tin topic
all_topic_info = topic_model.get_topic_info()

# 2. Loại bỏ topic -1 (outlier) để xem các chủ đề chính thống
filtered_info = all_topic_info[all_topic_info["Topic"] != -1]

print(f"Tổng số lượng chủ đề tìm thấy: {len(filtered_info)}")
print("-" * 100)
print(f"{'ID':<5} | {'Số lượng':<10} | {'Từ khóa mô tả chủ đề'}")
print("-" * 100)

for probs, row in filtered_info.iterrows():
    t_id = row["Topic"]
    t_count = row["Count"]
    
    # Lấy 10 từ khóa để có cái nhìn chi tiết nhất cho mỗi topic
    words = topic_model.get_topic(t_id)
    t_desc = ", ".join([w[0] for w in words[:10]])
    
    print(f"{t_id:<5} | {t_count:<10} | {t_desc}")

Tổng số lượng chủ đề tìm thấy: 37
----------------------------------------------------------------------------------------------------
ID    | Số lượng   | Từ khóa mô tả chủ đề
----------------------------------------------------------------------------------------------------
0     | 880        | data, experience, business, skills, ai, work, management, team, development, performance
1     | 747        | dữ_liệu, phân_tích, kinh_nghiệm, làm_việc, công_việc, để, khách_hàng, về, theo, phát_triển
2     | 63         | hitachi, people, best, innovation, work, digital, look, services, global, tomorrow
3     | 48         | binance, blockchain, trading, ecosystem, arrangement, web3, worlds, hiring, tools, financial
4     | 46         | community, exam, assessment, complete, media, content, internet, qualification, relevance, diverse
5     | 40         | publicis, consulting, possible, data, 60, candidates, team, equal, know, solutions
6     | 32         | qualcomm, engineering, research, rela

# Trực quan

In [8]:
# Trực quan hóa Top 15 topics với các từ khóa quan trọng nhất
fig_barchart = topic_model.visualize_barchart(top_n_topics=15, n_words=5)

# Hiển thị
fig_barchart.show()

# Lưu ra file HTML để xem trên trình duyệt
fig_barchart.write_html("../Figures/topic_distribution_barchart.html")

In [17]:
# Vẽ bản đồ khoảng cách giữa các Topic
fig_intertopic = topic_model.visualize_topics()

# Hiển thị
fig_intertopic.show()

fig_intertopic.write_html("../Figures/intertopic_distance_map.html")

In [16]:
# 1. Tính toán phân bổ theo nhãn chuyên môn
classes = df['Chuyên môn'].tolist()
topics_per_class = topic_model.topics_per_class(docs, classes=classes)

# 2. Trực quan hóa
fig_class = topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=10)

# Hiển thị
fig_class.show()

fig_class.write_html("../Figures/topics_by_job_role.html")